In [5]:
# importing the dataset

import pandas as pd

df=pd.read_csv('/content/SMSSpamCollection',sep='\t',names=['label','message'])

In [6]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
df.shape

(5572, 2)

In [8]:
df['message'].loc[100]

"Please don't text me anymore. I have nothing else to say."

In [9]:
# Data cleaning and preprocessing

import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [11]:
corpus=[]
for i in range(0, len(df)):
  review=re.sub('[^a-zA-Z]0-9',' ', df['message'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [12]:
corpus

['go jurong point, crazy.. avail bugi n great world la e buffet... cine got amor wat...',
 'ok lar... joke wif u oni...',
 "free entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question(std txt rate)t&c' appli 08452810075over18'",
 'u dun say earli hor... u c alreadi say...',
 'nah think goe usf, live around though',
 "freemsg hey darl 3 week' word back! i'd like fun still? tb ok! xxx std chg send, £1.50 rcv",
 'even brother like speak me. treat like aid patent.',
 "per request 'mell mell (oru minnaminungint nurungu vettam)' set callertun callers. press *9 copi friend callertun",
 'winner!! valu network custom select receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'mobil 11 month more? u r entitl updat latest colour mobil camera free! call mobil updat co free 08002986030',
 "i'm gonna home soon want talk stuff anymor tonight, k? i'v cri enough today.",
 'six chanc win cash! 100 20,000 pound txt> csh11 send 87

In [13]:
# Creating the bag of words model
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=2500,binary=True, ngram_range=(2,2))
X=cv.fit_transform(corpus).toarray()

In [14]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
X.shape

(5572, 2500)

In [16]:
y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [17]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [18]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [19]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [20]:
# Prediction

y_pred=spam_detect_model.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, classification_report

In [22]:
score=accuracy_score(y_test,y_pred)
score

0.9721973094170404

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       955
           1       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [24]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X=tv.fit_transform(corpus).toarray()

In [25]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [26]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [27]:
# prediction
y_pred=spam_detect_model.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score

0.9829596412556054

In [29]:
from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       955
           1       1.00      0.88      0.94       160

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



## word 2 vec

In [30]:
!pip install gensim==4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
# Creating model from scratch

In [32]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [33]:
nltk.download('wordnet')
nltk.download('omw-1.4')
corpus=[]
for i in range(0,len(df)):
  review=re.sub('[^a-zA-z0-9]',' ', df['message'][i])
  review=review.lower()
  review=review.split()
  review=[lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6days 16 tsandcs apply reply

In [35]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [36]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [37]:
nltk.download('punkt')
words=[]
for sent in corpus:
  sent_token=sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  're

In [39]:
import gensim

In [40]:
### lets train word2vec from scratch
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [41]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'free',
 'day',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'love',
 'text',
 'want',
 'send',
 'one',
 'need',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'dont',
 'reply',
 'think',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'please',
 'later',
 'pls',
 'co',
 'msg',
 'make',
 'dear',
 'night',
 'message',
 'well',
 'say',
 'min',
 'thing',
 'much',
 'claim',
 'great',
 'hope',
 'oh',
 'hey',
 'give',
 'number',
 'happy',
 'friend',
 'wat',
 'work',
 'yes',
 'way',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'ask',
 'win',
 'said',
 'cash',
 'life',
 'amp',
 'yeah',
 'im',
 'tone',
 'really',
 'meet',
 'babe',
 'find',
 'miss',
 'morning',
 'service',
 'thanks',
 'last',
 'uk',
 'com',
 'anything',
 'care',
 'would',
 'year',
 'also',
 'nokia',
 'lol',
 'feel',
 'every',
 'keep',
 'sure',
 'pick',
 'sent',
 'urgent',
 'contact',


In [42]:
model.corpus_count

5565

In [43]:
model.epochs

5

In [46]:
model.wv.similar_by_word('happy')

[('day', 0.9994611144065857),
 ('said', 0.99941486120224),
 ('year', 0.9993808269500732),
 ('amp', 0.9993687868118286),
 ('dont', 0.9993652701377869),
 ('dear', 0.9993575215339661),
 ('person', 0.9993318319320679),
 ('one', 0.9993273019790649),
 ('keep', 0.9993256330490112),
 ('make', 0.9993202090263367)]

In [47]:
model.wv['kid'].shape

(100,)

In [54]:
# avgword2vec

import numpy as np
def avg_word2vec(doc):
  # remove out-of-vocabulary words
  # sent = [word for word in doc if word in model.wv.index_to_key]
  # print(sent)

  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
          #or [np.zeros(len(model.wv.index_to_key))],axis=0)

In [49]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
from tqdm import tqdm

In [51]:
words[73]

['performed']

In [52]:
type(model.wv.index_to_key)

list

In [55]:
# apply for the entire scentence
X=[]
for i in tqdm(range(len(words))):
  X.append(avg_word2vec(words[i]))

  0%|          | 0/5565 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5565/5565 [00:01<00:00, 5297.88it/s]


In [56]:
X_new=np.array(X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [59]:
X_new[0].shape

(100,)

In [60]:
X_new

array([array([-0.130143  ,  0.29256067,  0.09476312,  0.00680268,  0.03676784,
              -0.36179435,  0.08419237,  0.52125937, -0.18404232, -0.07518957,
              -0.18611707, -0.33420444,  0.01759591,  0.04305888,  0.05434534,
              -0.24169992,  0.03093015, -0.3744651 , -0.07547817, -0.4127268 ,
               0.1613386 ,  0.0741827 ,  0.07036509, -0.17361031, -0.17242382,
              -0.00743655, -0.25074086, -0.22873528, -0.18533516,  0.09770682,
               0.26815683,  0.05375777,  0.12858109, -0.13609596, -0.12752658,
               0.2749419 , -0.03706096, -0.23745619, -0.15178245, -0.45032775,
               0.05146812, -0.29257807, -0.01763061,  0.07113836,  0.2058149 ,
              -0.05785598, -0.20306323, -0.04579333,  0.1560294 ,  0.17513962,
               0.15056647, -0.1944936 , -0.03305026, -0.03477367, -0.20189673,
               0.11648339,  0.1301739 ,  0.01120951, -0.3053696 ,  0.08349978,
               0.01302947,  0.11269622, -0.14919274,

In [61]:
### Assignment
# apply train test split and model